# 1. Assignment Solution (week4)

**[before]** charcater-level RNN

appl → pple

**[Assigment]** word-level RNN

Repeat is the best medicine for → is the best medicine for memory

- Hint: one-hot vector 대신 embedding 사용

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import ipdb

In [37]:
sentence = "Repeat is the best medicine for memory".split()
vocab = list(set(sentence))
print(vocab)


['medicine', 'Repeat', 'best', 'for', 'is', 'memory', 'the']


**Word2idx**

In [ ]:
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}     # 단어에 고유한 정수 부여
word2index['<unk>']=0
print(word2index)
print(word2index['memory'])

**Idx2word**

In [ ]:
index2word = {v: k for k, v in word2index.items()}
print(index2word)
print(index2word[2])

**Build Data**

In [ ]:
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence]    # word → index
    input_seq, label_seq = encoded[:-1], encoded[1:]    # input sequence, label sequence
    input_seq = torch.LongTensor(input_seq).unsqueeze(0)    # 차원 하나 추가 (for batch)
    label_seq = torch.LongTensor(label_seq).unsqueeze(0)    # 라벨의 차원 하나 추가(for batch)
    return input_seq, label_seq
X, Y = build_data(sentence, word2index)
print('input sequence:', X)    # (Repeat is the best medicine for)
print('label sequence:', Y)    # (is the best medicine for memory)

**Model**

In [ ]:
class Net(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(vocab_size, embedding_dim)     # for word embedding
        self.rnn_layer = nn.RNN(embedding_dim, hidden_dim, batch_first=batch_first)
        self.linear = nn.Linear(hidden_dim, vocab_size) # output size -> one-hot vector size (vocab size)

    def forward(self, x):    # (batch_size, sequence_length)
        # 1(Embedding Layer)
        output = self.embedding_layer(x)    # →(batch_size, sequence_length, embedding_dim)
        # 2(RNN Layer)
        output, hidden = self.rnn_layer(output)     # → output(batch_size, sequence_length, hidden_dim), hidden (1, batch_size, hidden_dim)
        # 3(최종 출력 Layer)
        output = self.linear(output)    # →(batch_size, sequence_length, vocab_size)
        return output.view(-1, output.size(2))  # batch 차원 제거 (batch_size*sequence_length, vocab_size)

In [ ]:
# hyperparameter
vocab_size = len(word2index)  # <unk> 토큰 포함
embedding_dim = 5
hidden_dim = 16

In [ ]:
model = Net(vocab_size, embedding_dim, hidden_dim, batch_first=True)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters())

In [ ]:
# 임의의 예측(random initialized weight)
output = model(X)
print(output)
print(output.shape)

**Train**

In [ ]:
for step in range(60):
    # gradient initialization
    optimizer.zero_grad()
    # forward
    output = model(X)
    # compute loss
    loss = loss_function(output, Y.view(-1))
    # backward
    loss.backward()
    # parameter update
    optimizer.step()
    if step % 5 == 0:
        print("[{:02d}/60] {:.4f} ".format(step+1, loss))
        pred = output.argmax(-1).tolist()
        print(" ".join(["Repeat"] + [index2word.get(x) for x in pred]))
        print()

# HuggingFace를 사용해 Transforemr 구현해보기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install transformers
!pip install datasets

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Trainer 클래스를 사용하지 않고, 2장과 같은 결과 얻는방법
* 다만, 섹션2에서 전처리를 완료했다고 가정
* 여기서 말하는 전처리는 Autotokenizer가 해주는 역할

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True) # map 함수는 datasets 라이브러리의 내장함수로, 정의한 tokenize_function을 입력으로 받아 효율적으로 데이터셋을 tokenize 할 수 있게 만들어준다.
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})


#1. Training을 위한 준비
2에서 나왔던 Trainer Class인 API를 이용하지 않고 2에서 와 같이 구현해야 하기 때문에, 몇가지를 새롭게 정의해야 한다. 첫 번째는 배치(batch)를 반복하는 데 사용할 dataloaders입니다. 그러나 이 dataloaders를 정의하기 전에 Trainer가 자동으로 수행한 몇 가지 작업을 직접 처리하기 위해 tokenized_datasets에 약간의 후처리를 적용해야 합니다. 구체적으로 다음을 수행해야 합니다:
  * 1. tokenized_datasets에 대한 후처리
    * 1.1 모델이 필요로 하지 않는 값이 저장된 열(columns)을 제거합니다. (sentence1, sentence2 등)
    * 1.2 열 레이블(column label)의 이름을 labels로 바꿉니다. 이는 모델이 labels라는 이름으로 매개변수를 받기 때문입니다.

    * 1.3 파이썬 리스트 대신 PyTorch 텐서(tensors)를 반환하도록 datasets의 형식을 설정합니다.
  * Autotokenizer의 역할중 encoding을 집적 해준다는 얘기이고, 이를 하는 이유는 model의 input으로 들어가는 조건을 맞추기 위해

tokenized_datasets에는 이러한 작업을 위한 별도의 메서드들이 존재합니다:

In [ ]:
print('######아무것도 안건들인 원본 dataset#####')
print(tokenized_datasets.column_names)
#1.1 모델이 필요로 하지 않는 값이 저장된 열(columns)을 제거합니다. (sentence1, sentence2 등)
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
#1.2 label의 이름을 labels로 바꿔준다
tokenized_datasets = tokenized_datasets.rename_column('label','labels')

print('################')
print(tokenized_datasets.column_names)

# datasets.set_format(): datasets()에서 제공해주는 함수 : dataset을 torch.tensor 형태로 바꿔줌
print('################')
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

print(tokenized_datasets)

######아무것도 안건들인 원본 dataset#####
{'train': ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'], 'validation': ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'], 'test': ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask']}
################
{'train': ['labels', 'input_ids', 'token_type_ids', 'attention_mask'], 'validation': ['labels', 'input_ids', 'token_type_ids', 'attention_mask'], 'test': ['labels', 'input_ids', 'token_type_ids', 'attention_mask']}
################
DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
}

위에서 보듯이 결과적으로 tokenized_datasets에는 모델이 허용하는 columns만으로 새롭게 구성했다(즉, Autotokenizer를 사용하지 않고서 만든 것)

이제 이 작업이 완료되었으므로 dataloader를 쉽게 정의할 수 있다: </br>
* Dataloader: https://tutorials.pytorch.kr/beginner/basics/data_tutorial.html

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets['train'],
    shuffle=True,
    batch_size=8, # batch_size = mini_batch = 8    =>  1번의 batch당 8개의 raw_data씩 처리
                  # train_data갯수(3668) / 8 = 459번의 batch 처리 해야함  => 1-epoch에 459번의 iteration
    collate_fn=data_collator
)

eval_dataloader = DataLoader(
    tokenized_datasets["validation"],
    batch_size=8,
    collate_fn=data_collator
)

데이터 처리에 오류가 없는지 빠르게 확인하기 위해 다음과 같이 배치(batch)를 검사할 수 있습니다:
* 1번째 방법. train_dataloader의 dictionary 형태를 이용해서 확인해보기
* 2번째 방법. Dataloader의 iter와 items

In [ ]:
# 1번째 방법. dictionary 형태를 이용해서 확인해보기
for one_batch in train_dataloader:
  # 한번의 batch만 포함시키기
  break
one_batch = {k: v for k,v in one_batch.items()}
one_batch
# token_type_ids : [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1] : "0": 첫번쨰 sequence의 token들 이다
#                                                                          "1": 두번째 sequence의 token들 이다
# 즉, 1raw당 2개의 sequence가 들어가 있다

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': tensor([1, 0, 1, 1, 1, 0, 1, 0]),
 'input_ids': tensor([[  101,  1000,  1996, 10984,  1997,  1031,  1996, 13612,  1033,  2003,
           7078, 26233,  2000,  2033,  1010,  1000,  7912, 11531,  2056,  1999,
           1037,  4861,  2000,  4419,  2739,  3149,  2197,  2733,  1012,   102,
           7912, 11531,  3843,  1037,  4861,  2197,  2733,  2000,  1996,  4419,
           2739,  3149,  3038,  1010,  1000,  1996, 10984,  1997,  1031,  1996,
          13612,  1033,  2003,  7078, 26233,  2000,  2033,  1012,   102,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0],
         [  101,  2012, 22878,  6928,  1010,  1996,  2184,  1011,  2095,  3602,
           2149, 10790, 22123,  1027, 25269,  2018,  5707,  2260,  1013,  3590,
           1999,  3976,  3228,  1037, 10750,  1997,  1017,  1012,  2385,  3867,
           2013,  1017,  1012,  2260,  3867,  1012,   102,  2582,  2041,  1996,
         

In [ ]:
# 2번째 방법. Dataloader의 iter()와 next()
data_iter = iter(train_dataloader)
print(next(data_iter))

{'labels': tensor([1, 0, 1, 0, 1, 1, 0, 1]), 'input_ids': tensor([[  101,  1999,  1037,  3661,  2741,  2000,  1996,  2335,  1998,  3191,
          2000,  1996,  3378,  2811,  2044,  2010,  8172,  1010, 10503, 11248,
          1000,  3167,  3314,  1000,  1998, 17806,  2005,  2010,  1000, 10876,
          2063,  1997,  4988,  2594, 11109,  1012,  1000,   102,  1999,  1037,
          3661,  2741,  2000,  1996,  2047,  2259,  2335,  2044,  2010,  8172,
          1010,  2720, 10503, 11248,  1000,  3167,  3314,  1000,  1998,  9706,
         12898, 17701,  2098,  2005,  2010,  1000, 10876,  2063,  1997,  4988,
          2594, 11109,  1000,  1012,   102],
        [  101,  2016,  2596,  1999,  2976,  2457,  2045,  6928,  1998,  2001,
          3517,  2000,  2022,  4015,  2000,  5395,  1999,  2048,  3134,  1012,
           102, 20977, 10795,  1999,  6044,  2006,  5958,  1998,  2001,  3517,
          2000,  2022,  4015,  2000,  5395,  1999,  2048,  3134,  1012,   102,
             0,     0,     0

In [ ]:
print(tokenizer.decode(one_batch['input_ids'][0]))

[CLS] " the timing of [ the miniseries ] is absolutely staggering to me, " nancy reagan said in a statement to fox news channel last week. [SEP] nancy reagan issued a statement last week to the fox news channel saying, " the timing of [ the miniseries ] is absolutely staggering to me. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


실제 데이터 형태(shapes)가 살짝 다를 수 있는데 이는 학습 dataloader에 대해 shuffle=True를 설정하고 배치(batch) 내에서의 최대 길이로 패딩(padding)하기 때문입니다.
  * 즉, iteration마다 data가 suffle되고 섞인것에 맞춰서 padding이 채워지니까

In [ ]:
#----------------------전처리 끝----------------------#

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


학습 과정에서 모든 것들이 원활하게 진행될 수 있는지 확인하기 위해 배치(batch)를 이 모델에 한번 전달해 봅시다:

In [ ]:
# 여기서는 one_batch만 들어가는 형태이다(즉, 모든 data에 대해서 loss를 구한게 아니라 minibatch=8에 대해서만 loss구한거임)
outputs = model(**one_batch) # **: data가 dict 형태니까 그 중에서 value값을 넣겠다
                             # * : dict형태에서 key값을 넣겠다
                             # 즉, 여기서는 label과 같이 들어갔으니까 loss도 같이 나온거임
print(outputs)
print(outputs.loss, outputs.logits.shape)



SequenceClassifierOutput(loss=tensor(0.7505, grad_fn=<NllLossBackward0>), logits=tensor([[0.4054, 0.0935],
        [0.4053, 0.0717],
        [0.4246, 0.0746],
        [0.4213, 0.0886],
        [0.4192, 0.0664],
        [0.3954, 0.0817],
        [0.4038, 0.0960],
        [0.4041, 0.1022]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor(0.7505, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

model의 parameter 확인 방법
* 모델의 파라미터를 확인 하는 방법은 model.parameters()를 통해 가능합니다. 단, model.parameters()는 generator 타입이므로 for문과 같이 순회하면서 또는 next를 이용하여 값을 접근할 수 있다.

In [ ]:
#1번째 방법: model.parameters()는 generator 형태이므로, iter형태이기 때문에 next()로 접근이 가능
print(next(model.parameters()))
print(next(model.parameters()).shape)

#2번째 방법: model.parameters()는 generator 형태이므로, iter형태이기 때문에 for문으로 접근 가능
print('#-------------------#')
for param in model.parameters():
  print(param)
  print(param.shape)
  break

Parameter containing:
tensor([[-0.0102, -0.0615, -0.0265,  ..., -0.0199, -0.0372, -0.0098],
        [-0.0117, -0.0600, -0.0323,  ..., -0.0168, -0.0401, -0.0107],
        [-0.0198, -0.0627, -0.0326,  ..., -0.0165, -0.0420, -0.0032],
        ...,
        [-0.0218, -0.0556, -0.0135,  ..., -0.0043, -0.0151, -0.0249],
        [-0.0462, -0.0565, -0.0019,  ...,  0.0157, -0.0139, -0.0095],
        [ 0.0015, -0.0821, -0.0160,  ..., -0.0081, -0.0475,  0.0753]],
       requires_grad=True)
torch.Size([30522, 768])
#-------------------#
Parameter containing:
tensor([[-0.0102, -0.0615, -0.0265,  ..., -0.0199, -0.0372, -0.0098],
        [-0.0117, -0.0600, -0.0323,  ..., -0.0168, -0.0401, -0.0107],
        [-0.0198, -0.0627, -0.0326,  ..., -0.0165, -0.0420, -0.0032],
        ...,
        [-0.0218, -0.0556, -0.0135,  ..., -0.0043, -0.0151, -0.0249],
        [-0.0462, -0.0565, -0.0019,  ...,  0.0157, -0.0139, -0.0095],
        [ 0.0015, -0.0821, -0.0160,  ..., -0.0081, -0.0475,  0.0753]],
       require

마지막으로, Trainer에서 디폴트로 사용되는 학습률 스케줄러(learning rate scheduler)는 최대값(5e-5)에서 0까지 선형 감쇠(linear decay)합니다. 이를 적절하게 정의하려면 우리가 수행할 학습 단계의 횟수를 알아야 합니다. 이는 실행하려는 에포크(epochs) 수에 학습 배치(batch)의 개수를 곱한 것입니다. 학습 배치의 개수는 학습 dataloader의 길이와 같습니다. Trainer는 디폴트로 3개의 에포크(epochs)를 사용하므로 다음을 따릅니다:
* 1epoch에 batch_size만큼의 weight이 update, 전체적으로 보면 weight은 1번씩 update된것
* batch의 갯수 = dataloader의 길이(len(train_dataloader))
* warmup_step : learning rate가 0.01이라고 한다면 처음 10 step 동안은 0.001, 0.002, 0.003 ~ 0.01까지 선형적으로 조금씩만 증가하는 learning rate을 사용하는 것입니다. 이는 반대로 말하면 샘플이 적은 초기에 아주아주 작은 learning rate을 사용함으로써 bad local optima 로의 학습이 일어나지 않게 만든다
  * https://zzaebok.github.io/deep_learning/RAdam/

In [ ]:
print(len(train_dataloader)) #batch의 갯수
                             # 즉, 459이면 batch의 갯수가 459이다
                             # 즉 , batch를 459번 반복한다 = 459 iteration = 1epoch
                             # 여기선 batch_size(mini_batch) = 8 이었고, train_data=3668 이니까
                             # 3668/8(batch_size) = 459 = 459번의 batch = 459 iteration

459


In [ ]:
from transformers import get_scheduler

num_epochs = 3 #train_Data전체를 3 epoch으로 train
num_training_steps = num_epochs * len(train_dataloader) # len(train_dataloader) : 1epoch당 batch의 갯수
                                                        # 1377
lr_scheduler = get_scheduler(
    'linear', # The name of the scheduler to use.
    optimizer = optimizer,
    num_warmup_steps=0,            #Q)warmup_step을 이용해 초반의 step 동안은 0.001, 0.002, 0.003 ~ 0.01까지 linearly하게 증가하게 사용한다. 이값을 설정하는게 어떤 의미가 있을까?
                                   # Hugging face에서는 warmup step의 수라고 한다..
                                   # 그렇다면, num_warmup_steps=10이라면, 10 step동안은 Linearly하게 learning rate를 Linearly하게 증가시킨 후 .. 일정하게 사용하는 느낌?
                                   # Reference : https://huggingface.co/docs/transformers/main_classes/optimizer_schedules
    num_training_steps=num_training_steps # 1377
)

#2. Training Loop

model과 배치(batch)를 적재할 device를 정의한다.

In [ ]:
import torch

#device = torch.device("cuda:0")  # 0번쨰 gpu로 device를 사용하겠다
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device) #설정된 device에 model할당
device

device(type='cuda')

학습이 언제 끝날지 정보를 얻기 위해 tqdm 라이브러리를 사용하여 학습 단계(training steps)를 기준으로 진행 표시줄(progress bar)을 출력할 수 있도록 합니다:
https://gaussian37.github.io/dl-pytorch-snippets/#optimizerzero_grad-lossbackward-optimizerstep-1 참고

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train() # model.train() : 단지 model한테 학습할거라고 알려주는 것 뿐이다
'''
#앞전에 정의했던 dataloader
train_dataloader = DataLoader(
    tokenized_datasets['train'],
    shuffle=True,
    batch_size=8, # batch_size = mini_batch = 8    =>  1번의 batch당 8개의 raw_data씩 처리
                  # train_data갯수(3668) / 8 = 459번의 batch 처리 해야함  => 1-epoch에 459번의 iteration
    collate_fn=data_collator
)
'''

for epoch in range(num_epochs):
    #1번의 epoch에 459번의 batch처리
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad() # batch가 끝나고, 다시 반복할대마다 새로운 gradient를 계산해야 하므로, 새롭게 초기화
        progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

# Evaluation Loop


In [ ]:
from datasets import load_metric

metric = load_metric("glue", "mrpc") # evaluation metric

model.eval() # model을 evaluation 모드로 변환
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

# evaluation 결과계산 및 출력
metric.compute()

<ipython-input-36-5fc8e56794ae>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "mrpc") # evaluation metric


{'accuracy': 0.8602941176470589, 'f1': 0.9015544041450777}

evaluation시, with torch.no_grad() 쓰는 이유:  https://coffeedjimmy.github.io/pytorch/2019/11/05/pytorch_nograd_vs_train_eval/
